In [1]:
#Загрузка файла с табличными данными
!wget https://www.dropbox.com/s/2yjaru8mbkeynwe/dataset%20%283%29.csv?dl=1 -O dataset.csv

--2020-05-20 13:25:43--  https://www.dropbox.com/s/2yjaru8mbkeynwe/dataset%20%283%29.csv?dl=1
Resolving www.dropbox.com... 162.125.64.1, 2620:100:6020:1::a27d:4001
Connecting to www.dropbox.com|162.125.64.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/2yjaru8mbkeynwe/dataset%20%283%29.csv [following]
--2020-05-20 13:25:43--  https://www.dropbox.com/s/dl/2yjaru8mbkeynwe/dataset%20%283%29.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccd2a1b7789d31309ca5f04670f.dl.dropboxusercontent.com/cd/0/get/A4FqtYhLPIuZzMilSjeu_8jNlFHyJpq6KvMmSpz501aKgJOpjnAL9hbQX1I9ADV7Bpl0mURnAIksuPE6m1jt7ZgfTStlsomhmxyK66cyk4pT1A4WVqPP_EKD_16hbpgipks/file?dl=1# [following]
--2020-05-20 13:25:44--  https://uccd2a1b7789d31309ca5f04670f.dl.dropboxusercontent.com/cd/0/get/A4FqtYhLPIuZzMilSjeu_8jNlFHyJpq6KvMmSpz501aKgJOpjnAL9hbQX1I9ADV7Bpl0mURnAIksuPE6m1jt7ZgfTStlsomhmxyK66cyk4pT1A4WVqPP_E

In [3]:
#Импортирование необходимых библиотек для обработки и чтение файла с табличными данными
import cv2
import os
import numpy as np
import shutil
import torch
import zipfile
import cv2
import os
import numpy as np
import pandas as pd
import datetime as dt
dataset = pd.read_csv('dataset.csv')

In [4]:
#Обработка временных показателей даты загрузки видео на видеохостинг и даты создания канала.
dataset['Time_cur'] = pd.to_datetime(dataset['published_at'])
dataset['published_hour_cur'] = dataset['Time_cur'].dt.hour
dataset['Weekday_cur'] = dataset['Time_cur'].dt.weekday
diff = dt.datetime.now() - dataset['Time_cur']
dataset['Published_cur_time'] = (dt.datetime.now() - dataset['Time_cur']).dt.total_seconds()
dataset  = dataset.drop(['Time_cur', 'published_at'], axis=1)
dataset['channel_founded'] = pd.to_datetime(dataset['channel_founded'])
diff = dt.datetime.now() - dataset['channel_founded']
dataset['channel_founded'] = diff.dt.total_seconds()

In [5]:
# Деление видео на классы популярных и непопулярных
dataset['popular'] = 0.0
dataset['popular'][dataset.view_count_new.gt(dataset.view_count_new.quantile(0.8))] = 1.0

/anaconda/envs/py37_default/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
# Обработка категориальных признаков(день недели и час публикации видео) при помощи one hot encoder.
from sklearn.preprocessing import OneHotEncoder
dataset = pd.get_dummies(dataset, columns=['published_hour_cur', 'Weekday_cur'], prefix=["published_hour", "Weekday_cur"] )

In [7]:
# Приминение нормализации к вещественным признакам, которые используются для классификации видео.
from sklearn.preprocessing import StandardScaler
dataset_norm = dataset[['channel_founded', 'subcriber_count', 'view_count', 'like_count', 'dislike_count', 'favorite_count', 'comment_count',  'Published_cur_time' ]]
dataset_norm  = dataset_norm.values
scaler = StandardScaler()
dataset_norm = scaler.fit_transform(dataset_norm)
dataset[['channel_founded', 'subcriber_count', 'view_count', 'like_count', 'dislike_count', 'favorite_count', 'comment_count',  'Published_cur_time' ]] = dataset_norm 







In [ ]:
# Создание класса data.dataset, который необходим для загрузки данных в модель MLP.

In [13]:
from torch.utils import data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
class numerical(data.Dataset):
    def __init__(self, numerical, cat, y):
        "Initialization"
        self.numerical = numerical
        self.cat = cat
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def read_images_folder(self, folder):
        frames_vid = os.listdir(os.path.join(self.data_path_frame, folder))
        def myFunc(e):
            return int(e.split('.')[0])
        out = []
        frames_vid.sort(reverse=False, key=myFunc)
        for pic in frames_vid:
            image = Image.open(os.path.join(self.data_path_frame, folder, pic))
            if self.transform_frames:
                image = self.transform_frames(image)
            out.append(image)
        out = torch.stack(out, dim = 0)
        return out
    
    def __getitem__(self, index):
        num = torch.from_numpy(self.numerical[index])
        cat = torch.from_numpy(self.cat[index])
        y = list(self.y)[index]
        return num, cat, y

In [14]:
#Разбиение набора данных на обучающую, валидационную и тестовую выборку.

In [15]:
train_ids = int(0.8*len(dataset))
val_ids = int(0.8*len(dataset)) + int(0.1*len(dataset))
train = dataset[:train_ids]
val = dataset[train_ids:val_ids]
test = dataset[val_ids:len(dataset)]

In [16]:
train_numerical = train[['channel_founded', 'subcriber_count', 'view_count', 'like_count','dislike_count', 'comment_count', 'Published_cur_time']].values
train_categorical = train[['published_hour_0', 'published_hour_1', 'published_hour_2',
       'published_hour_3', 'published_hour_4', 'published_hour_5',
       'published_hour_6', 'published_hour_7', 'published_hour_8',
       'published_hour_9', 'published_hour_10', 'published_hour_11',
       'published_hour_12', 'published_hour_13', 'published_hour_14',
       'published_hour_15', 'published_hour_16', 'published_hour_17',
       'published_hour_18', 'published_hour_19', 'published_hour_20',
       'published_hour_21', 'published_hour_22', 'published_hour_23',
       'Weekday_cur_0', 'Weekday_cur_1', 'Weekday_cur_2', 'Weekday_cur_3',
       'Weekday_cur_4', 'Weekday_cur_5', 'Weekday_cur_6']].values
train_target = train[['popular']].values

In [17]:
val_numerical = val[['channel_founded', 'subcriber_count', 'view_count', 'like_count','dislike_count', 'comment_count', 'Published_cur_time']].values
val_categorical = val[['published_hour_0', 'published_hour_1', 'published_hour_2',
       'published_hour_3', 'published_hour_4', 'published_hour_5',
       'published_hour_6', 'published_hour_7', 'published_hour_8',
       'published_hour_9', 'published_hour_10', 'published_hour_11',
       'published_hour_12', 'published_hour_13', 'published_hour_14',
       'published_hour_15', 'published_hour_16', 'published_hour_17',
       'published_hour_18', 'published_hour_19', 'published_hour_20',
       'published_hour_21', 'published_hour_22', 'published_hour_23',
       'Weekday_cur_0', 'Weekday_cur_1', 'Weekday_cur_2', 'Weekday_cur_3',
       'Weekday_cur_4', 'Weekday_cur_5', 'Weekday_cur_6']].values
val_target = val[['popular']].values

In [18]:
test_numerical = test[['channel_founded', 'subcriber_count', 'view_count', 'like_count','dislike_count', 'comment_count', 'Published_cur_time']].values
test_categorical = test[['published_hour_0', 'published_hour_1', 'published_hour_2',
       'published_hour_3', 'published_hour_4', 'published_hour_5',
       'published_hour_6', 'published_hour_7', 'published_hour_8',
       'published_hour_9', 'published_hour_10', 'published_hour_11',
       'published_hour_12', 'published_hour_13', 'published_hour_14',
       'published_hour_15', 'published_hour_16', 'published_hour_17',
       'published_hour_18', 'published_hour_19', 'published_hour_20',
       'published_hour_21', 'published_hour_22', 'published_hour_23',
       'Weekday_cur_0', 'Weekday_cur_1', 'Weekday_cur_2', 'Weekday_cur_3',
       'Weekday_cur_4', 'Weekday_cur_5', 'Weekday_cur_6']].values
test_target = test[['popular']].values

In [19]:
dataset_train = numerical(train_numerical, train_categorical, train_target)

In [20]:
dataset_val = numerical(val_numerical, val_categorical, val_target)

In [21]:
#Инициализация data.DataLoader, который позволяет передавать данные по батчам в нейронную сеть.
train_dl = data.DataLoader(dataset_train, batch_size = 400)
val_dl = data.DataLoader(dataset_val, batch_size = 400)

In [23]:
# Класс нейронной сети
class numerical_categ(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.numer_1 = nn.Linear(7, 16)
        self.funcnumer_1 = nn.LeakyReLU(0.1)
        self.numer_2 = nn.Linear(16, 32)
        
        self.categ_1 = nn.Linear(31, 32)
        self.funccateg_1 = nn.LeakyReLU(0.1)
        self.categ_2 = nn.Linear(32, 32)
        
        self.comb1 = nn.Linear(64, 32)
        self.dropout = nn.Dropout(0.4)
        self.funccateg_1 = nn.LeakyReLU(0.1)
        self.comb2 = nn.Linear(32, 1)
        
    def forward(self, categ, numerical):
        num = self.numer_1(numerical)
        num = self.funcnumer_1(num)
        num = self.numer_2(num)
        
        categ = self.categ_1(categ)
        categ = self.funccateg_1(categ)
        categ = self.categ_2(categ)
        
        comb = torch.cat([categ, num], axis = -1)
        comb = self.comb1(comb)
        comb = self.dropout(comb)
        comb = self.funccateg_1(comb)
        comb = self.comb2(comb)
        return comb
        

In [24]:
# Подсчет точности предсказаний
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    acc = acc.item()
    return acc

In [25]:
model = torch.nn.DataParallel(numerical_categ().cuda())

In [26]:
from torch.utils.tensorboard import SummaryWriter


In [27]:
#Функция обучения модели
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
train_log = []
val_log = []
import matplotlib.pyplot as plt
def train_model(model, epoch, lr):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    opt_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5)
    opt_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'max', patience =  1, factor=0.1)
    criterion = nn.BCEWithLogitsLoss()
    writer = SummaryWriter()
    for i in range(epoch):
        model.train()
        sum_loss = 0.0
        total = 0
        train_log_epoch = []
        train_val_epoch = []
        num_batch = 0
        total_predicts = []
        total_corrects = []
        
        for elem in train_dl:
            optimizer.zero_grad()
            num = elem[0].float().cuda()
            cat = elem[1].float().cuda()
            y = elem[2].float().cuda()
            y_pred = model(cat, num)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            loss = loss.item()
            acc = binary_acc(y_pred, y)
            train_log_epoch.append(acc)
            y_pred_sig = torch.sigmoid(y_pred)
            y_pred_sig = y_pred_sig.flatten().tolist()
            total_predicts.extend(y_pred_sig)
            total_corrects.extend(y.flatten().tolist())
        train_log.extend(train_log_epoch)
        acc = np.mean(train_log_epoch)
        binary_preds = np.array(total_predicts)>0.5
        binary_preds = binary_preds.astype(int)
        binary_preds = list(binary_preds)
        print(len(binary_preds))
        print(len(total_corrects))
        torch.save(model.state_dict(), 'checkpoint/model_numer'+str(i)+'.pth')
        prec = precision_score(total_corrects, binary_preds)
        rec = recall_score(total_corrects, binary_preds)
        roc_auc = roc_auc_score(total_corrects, total_predicts)
        writer.add_scalar('precision_train', prec, i)
        writer.add_scalar('recall_train', rec, i)
        writer.add_scalar('roc_auc_train', roc_auc, i)
        writer.add_scalar('accuracy_train', np.mean(train_log_epoch), i)
        print(prec)
        print(rec)
        print(roc_auc)
        print(acc)
        
        print('test')
        prec_test, rec_test, roc_auc_test, acc_test = val(model, val_dl)
        writer.add_scalar('precision_test', prec_test, i)
        writer.add_scalar('recall_test', rec_test, i)
        writer.add_scalar('roc_auc_test', roc_auc_test, i)
        writer.add_scalar('accuracy_test', acc_test, i)
        opt_scheduler.step(acc_test)
        print(prec_test)
        print(rec_test)
        print(roc_auc_test)
        print(acc_test)
        writer.close()
    return model


def val(model, valid_dl):
    model.eval()
    sum_loss = 0.0
    val_epoch = []
    total_preds = []
    total_corrects = []
    for elem in valid_dl:
        num = elem[0].float().cuda()
        cat = elem[1].float().cuda()
        y = elem[2].float().cuda()
        y_pred = model(cat, num)
        y_pred_sig = torch.sigmoid(y_pred)
        y_pred_sig = y_pred_sig.flatten().tolist()
        total_preds.extend(y_pred_sig)
        total_corrects.extend(y.flatten().tolist())
        acc = binary_acc(y_pred, y)
        val_epoch.append(acc)
    binary_preds = np.array(total_preds)>0.5
    binary_preds = binary_preds.astype(int)
    binary_preds = list(binary_preds)
    prec = precision_score(total_corrects, binary_preds)
    rec = recall_score(total_corrects, binary_preds)
    roc_auc = roc_auc_score(total_corrects, total_preds)
    acc = np.mean(val_epoch)
    return prec, rec, roc_auc, acc

In [28]:
model = train_model(model, 30, 0.01)

8278
8278
0.6424581005586593
0.06894484412470024
0.5349415172854153
80.23809523809524
test
0.5555555555555556
0.189873417721519
0.7786424831514805
79.33333333333333
8278
8278
0.7
0.16786570743405277
0.7294647489270307
81.52380952380952
test
0.6016949152542372
0.29957805907172996
0.798373648015501
80.66666666666667
8278
8278
0.7073509015256588
0.3057553956834532
0.7832059465891734
83.28571428571429
test
0.5684210526315789
0.45569620253164556
0.8149706970760604
80.33333333333333
8278
8278
0.6905241935483871
0.4106714628297362
0.8075335495597471
84.38095238095238
test
0.5894736842105263
0.47257383966244726
0.8319224518103225
80.66666666666667
8278
8278
0.6994219653179191
0.4352517985611511
0.8349369369859634
84.52380952380952
test
0.5869565217391305
0.45569620253164556
0.8332989215888696
80.33333333333333
8278
8278
0.6999012833168805
0.4250599520383693
0.8375857559035979
84.57142857142857
test
0.5919540229885057
0.4345991561181435
0.8343365680373128
80.66666666666667
8278
8278
0.701960784

In [31]:
dataset_test = numerical(test_numerical, test_categorical, test_target)
test_dl = data.DataLoader(dataset_test, batch_size = 32)

In [32]:
prec, rec, roc_auc, acc = val(model, test_dl)

In [36]:
print(prec, rec, roc_auc, acc)

0.5161290322580645 0.3878787878787879 0.8075983717774763 84.81818181818181


In [ ]:
def binary_acc(y_pred, y_test, thresh):
    y_pred_tag = torch.gt(torch.sigmoid(y_pred), thresh).int()
    y_pred_tag = y_pred_tag
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    print(correct_results_sum)
    acc = correct_results_sum/y_test.shape[0]
    acc = acc.item()
    return acc


In [ ]:
from tqdm import tqdm
model.eval()
sum_loss = 0.0
val_epoch = []
total_preds = []
total_corrects = []
criterion = nn.BCEWithLogitsLoss()
thresholds = np.arange(0, 1, 0.05)
for thresh in tqdm(thresholds):
    for elem in val_dl:
        num = elem[0].float().cuda()
        cat = elem[1].float().cuda()
        y = elem[2].float().cuda()
        y_pred = model(cat, num)
        y_pred_sig = torch.sigmoid(y_pred)
        y_pred_sig = y_pred_sig.flatten().tolist()
        total_preds.extend(y_pred_sig)
        total_corrects.extend(y.flatten().tolist())
        acc = binary_acc(y_pred, y, thresh)
    val_epoch.append(acc)
    binary_preds = np.array(total_preds)>thresh
    binary_preds = binary_preds.astype(int)
    binary_preds = list(binary_preds)
    print('test')
    prec = precision_score(total_corrects, binary_preds)
    rec = recall_score(total_corrects, binary_preds)
    roc_auc = roc_auc_score(total_corrects, total_preds)
    acc = np.mean(val_epoch)
    print('thresh', thresh, 'prec', prec, 'rec', rec, 'roc_auc', roc_auc, 'acc', acc  )
    

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
out = precision_recall_curve(total_corrects, total_preds)

In [ ]:
out = precision_recall_curve(total_corrects, total_preds)
import matplotlib.pyplot as plt
precision, recall, thrs = out
plt.plot(recall, precision)




In [ ]:
!pip install xgboost

In [ ]:
train_numerical = train[['channel_founded', 'subcriber_count', 'view_count', 'like_count','dislike_count', 'comment_count', 'Published_cur_time']]
train_categorical = train[['published_hour_0', 'published_hour_1', 'published_hour_2',
       'published_hour_3', 'published_hour_4', 'published_hour_5',
       'published_hour_6', 'published_hour_7', 'published_hour_8',
       'published_hour_9', 'published_hour_10', 'published_hour_11',
       'published_hour_12', 'published_hour_13', 'published_hour_14',
       'published_hour_15', 'published_hour_16', 'published_hour_17',
       'published_hour_18', 'published_hour_19', 'published_hour_20',
       'published_hour_21', 'published_hour_22', 'published_hour_23',
       'Weekday_cur_0', 'Weekday_cur_1', 'Weekday_cur_2', 'Weekday_cur_3',
       'Weekday_cur_4', 'Weekday_cur_5', 'Weekday_cur_6']]
train_target = train[['popular']]

In [ ]:
train_numerical = train_numerical.rename(columns={"view_count": "view_count_prev", "like_count": "like_count_prev", "dislike_count":"dislike_count_prev", "comment_count":"comment_count_prev" })

In [ ]:
train_boost = [train_numerical, train_categorical]
train_boost = pd.concat(train_boost, axis = 1)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf =  GradientBoostingClassifier(random_state=0)
clf.fit(train_boost, train_target.values.ravel())

In [ ]:
val_numerical = val[['channel_founded', 'subcriber_count', 'view_count', 'like_count','dislike_count', 'comment_count', 'Published_cur_time']]
val_categorical = val[['published_hour_0', 'published_hour_1', 'published_hour_2',
       'published_hour_3', 'published_hour_4', 'published_hour_5',
       'published_hour_6', 'published_hour_7', 'published_hour_8',
       'published_hour_9', 'published_hour_10', 'published_hour_11',
       'published_hour_12', 'published_hour_13', 'published_hour_14',
       'published_hour_15', 'published_hour_16', 'published_hour_17',
       'published_hour_18', 'published_hour_19', 'published_hour_20',
       'published_hour_21', 'published_hour_22', 'published_hour_23',
       'Weekday_cur_0', 'Weekday_cur_1', 'Weekday_cur_2', 'Weekday_cur_3',
       'Weekday_cur_4', 'Weekday_cur_5', 'Weekday_cur_6']]
val_target = val[['popular']].values

In [ ]:
val_numerical = val_numerical.rename(columns={"view_count": "view_count_prev", "like_count": "like_count_prev", "dislike_count":"dislike_count_prev", "comment_count":"comment_count_prev" })

In [ ]:
val_boost = [val_numerical, val_categorical]
val_boost = pd.concat(val_boost, axis = 1)

In [ ]:
feature_importance = clf.feature_importances_

In [ ]:
feature_importance[sorted_idx][10:]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
sorted_idx = np.argsort(feature_importance)
pos = np.arange(10) + .5
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx][-10:], align='center')
plt.yticks(pos, np.array(train_boost.columns)[sorted_idx][-10:])
plt.title('Feature Importance')

result = permutation_importance(clf, val_boost, val_target, n_repeats=10,
                                random_state=42, n_jobs=2)
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(result.importances[sorted_idx][-10:].T,
            vert=False, labels=np.array(val_boost.columns)[sorted_idx][-10:])
plt.title("Permutation Importance (test set)")
fig.tight_layout()
plt.show()

In [ ]:
res = clf.predict_proba(val_boost)

In [ ]:
res = res[:,1]

In [ ]:
res

In [ ]:
prec = precision_score(val_target, list(np.round(res)))
rec = recall_score(val_target, list(np.round(res)))
roc_auc = roc_auc_score(val_target, res)

# Gradboost

In [ ]:
test_numerical =  test[['channel_founded', 'subcriber_count', 'view_count', 'like_count','dislike_count', 'comment_count', 'Published_cur_time']]
test_categorical = test[['published_hour_0', 'published_hour_1', 'published_hour_2',
       'published_hour_3', 'published_hour_4', 'published_hour_5',
       'published_hour_6', 'published_hour_7', 'published_hour_8',
       'published_hour_9', 'published_hour_10', 'published_hour_11',
       'published_hour_12', 'published_hour_13', 'published_hour_14',
       'published_hour_15', 'published_hour_16', 'published_hour_17',
       'published_hour_18', 'published_hour_19', 'published_hour_20',
       'published_hour_21', 'published_hour_22', 'published_hour_23',
       'Weekday_cur_0', 'Weekday_cur_1', 'Weekday_cur_2', 'Weekday_cur_3',
       'Weekday_cur_4', 'Weekday_cur_5', 'Weekday_cur_6']]
test_target = test[['popular']].values

In [ ]:
test_boost = [test_numerical, test_categorical]
test_boost = pd.concat(test_boost, axis = 1)

In [ ]:
#lr = 0.1
#n_est = 100
#max_depth = 3
clf =  GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, max_depth=3)
clf.fit(train_boost, train_target.values.ravel())


In [ ]:
res = clf.predict_proba(val_boost)

In [ ]:
def binary_acc(y_pred, y_test, thresh):
    y_pred_tag = (y_pred>thresh).astype(int)
    print(y_pred_tag.shape, y_test.shape)
    correct_results_sum = (y_pred_tag == y_test.squeeze()).sum()
    print(correct_results_sum)
    acc = correct_results_sum/y_test.shape[0]
    acc = acc.item()
    return acc

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
prec = precision_score(val_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(val_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(val_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), val_target.squeeze().astype(int), 0.5))

In [ ]:
res = clf.predict_proba(test_boost)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
prec = precision_score(test_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(test_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(test_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), test_target.squeeze().astype(int), 0.5))

In [ ]:
#lr = 0.1
#n_est = 1000
#max_depth = 3
clf =  GradientBoostingClassifier(learning_rate=0.1, n_estimators=1000, max_depth=3)
clf.fit(train_boost, train_target.values.ravel())

In [ ]:
res = clf.predict_proba(val_boost)

In [ ]:
prec = precision_score(val_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(val_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(val_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), val_target.squeeze().astype(int), 0.5))

In [ ]:
res = clf.predict_proba(test_boost)

In [ ]:
prec = precision_score(test_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(test_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(test_target.squeeze().astype(int), np.round(res[:,1]))


In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), test_target.squeeze().astype(int), 0.5))

In [ ]:
#lr = 0.1
#n_est = 100
#max_depth = 3
clf =  GradientBoostingClassifier(learning_rate=0.1, n_estimators=10000, max_depth=3)
clf.fit(train_boost, train_target.values.ravel())

In [ ]:
res = clf.predict_proba(val_boost)

In [ ]:
prec = precision_score(val_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(val_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(val_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), val_target.squeeze().astype(int), 0.5))

In [ ]:
res = clf.predict_proba(test_boost)

In [ ]:
prec = precision_score(test_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(test_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(test_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), test_target.squeeze().astype(int), 0.5))

In [ ]:
clf =  GradientBoostingClassifier(learning_rate=0.01, n_estimators=100, max_depth=3)
clf.fit(train_boost, train_target.values.ravel())

In [ ]:
res = clf.predict_proba(val_boost)

In [ ]:
prec = precision_score(val_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(val_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(val_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), val_target.squeeze().astype(int), 0.5))

In [ ]:
res = clf.predict_proba(test_boost)

In [ ]:
prec = precision_score(test_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(test_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(test_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), test_target.squeeze().astype(int), 0.5))

In [ ]:
clf =  GradientBoostingClassifier(learning_rate=0.001, n_estimators=100, max_depth=3)
clf.fit(train_boost, train_target.values.ravel())

In [ ]:
res = clf.predict_proba(val_boost)

In [ ]:
prec = precision_score(val_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(val_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(val_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), val_target.squeeze().astype(int), 0.5))

In [ ]:
res = clf.predict_proba(test_boost)

In [ ]:
prec = precision_score(test_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(test_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(test_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), test_target.squeeze().astype(int), 0.5))

In [ ]:
clf =  GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, max_depth=4)
clf.fit(train_boost, train_target.values.ravel())

In [ ]:
res = clf.predict_proba(val_boost)

In [ ]:
prec = precision_score(val_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(val_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(val_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), val_target.squeeze().astype(int), 0.5))

In [ ]:
res = clf.predict_proba(test_boost)

In [ ]:
prec = precision_score(test_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(test_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(test_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), test_target.squeeze().astype(int), 0.5))

In [ ]:
clf =  GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, max_depth=1)
clf.fit(train_boost, train_target.values.ravel())

In [ ]:
res = clf.predict_proba(val_boost)

In [ ]:
prec = precision_score(val_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(val_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(val_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), val_target.squeeze().astype(int), 0.5))

In [ ]:
res = clf.predict_proba(test_boost)

In [ ]:
prec = precision_score(test_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(test_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(test_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), test_target.squeeze().astype(int), 0.5))

In [ ]:
clf =  GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, max_depth=2)
clf.fit(train_boost, train_target.values.ravel())

In [ ]:
res = clf.predict_proba(val_boost)
prec = precision_score(val_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(val_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(val_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), val_target.squeeze().astype(int), 0.5))

In [ ]:
res = clf.predict_proba(test_boost)
prec = precision_score(test_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(test_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(test_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), test_target.squeeze().astype(int), 0.5))

In [ ]:
clf =  GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, max_depth=5)
clf.fit(train_boost, train_target.values.ravel())

In [ ]:
res = clf.predict_proba(val_boost)
prec = precision_score(val_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(val_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(val_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), val_target.squeeze().astype(int), 0.5))

In [ ]:
res = clf.predict_proba(test_boost)
prec = precision_score(test_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(test_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(test_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), test_target.squeeze().astype(int), 0.5))

In [ ]:
clf =  GradientBoostingClassifier(learning_rate=0.01, n_estimators=10000, max_depth=1)
clf.fit(train_boost, train_target.values.ravel())

In [ ]:
res = clf.predict_proba(val_boost)
prec = precision_score(val_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(val_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(val_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), val_target.squeeze().astype(int), 0.5))

In [ ]:
res = clf.predict_proba(test_boost)
prec = precision_score(test_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(test_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(test_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), test_target.squeeze().astype(int), 0.5))

In [ ]:
clf =  GradientBoostingClassifier(learning_rate=0.01, n_estimators=10000, max_depth=2)
clf.fit(train_boost, train_target.values.ravel())

In [ ]:
res = clf.predict_proba(val_boost)
prec = precision_score(val_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(val_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(val_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), val_target.squeeze().astype(int), 0.5))

In [ ]:
res = clf.predict_proba(test_boost)
prec = precision_score(test_target.squeeze().astype(int), np.round(res[:,1]))
rec = recall_score(test_target.squeeze().astype(int), np.round(res[:,1]))
roc_auc = roc_auc_score(test_target.squeeze().astype(int), np.round(res[:,1]))

In [ ]:
print(prec)
print(rec)
print(roc_auc)

In [ ]:
print(binary_acc(np.round(res[:,1]), test_target.squeeze().astype(int), 0.5))

In [ ]:
import matplotlib.pyplot as plt
out = precision_recall_curve(test_target, res)
precision, recall, thrs = out
plt.plot(recall, precision)

In [ ]:
thresholds = np.arange(0, 1, 0.05)
for thresh in tqdm(thresholds):
    y_bin = res > thresh
    y_bin = y_bin.astype(int)

    acc = binary_acc(res, val_target, thresh)
    prec = precision_score(val_target, y_bin)
    rec = recall_score(val_target, y_bin)
    roc_auc = roc_auc_score(val_target, res)
    print('thresh', thresh, 'prec', prec, 'rec', rec, 'roc_auc', roc_auc, 'acc', acc  )
